# Tema 7. PSI-BLAST

Quan se seqüencien els mRNA d'un organisme, es genera un **transcriptoma**: la col·lecció de seqüències dels gens expressats als teixits que formaven part de la mostra. A la base de dades *European Nucleotide Archive* (ENA) podem trobar molts transcriptomes assemblats. Típicament, després d'assemblar un transcriptoma cal determinar la funció de cada trànscrit. Els trànscrits codificants de proteïnes són traduïts i les seqüències proteiques es comparen amb les de les bases de dades. Les bases de dades més útils per identificar la funció d'aquests trànscrits són aquelles que tenen major quantitat d'informació funcional: Pfam, Swissprot, KEGG, etc.

Per a aquesta pràctica, hem pres un transcriptoma de l'escarabat *Ips typographus*, amb nombre d'accés GACR01000000 en ENA. Dels 14689 trànscrits, se n'han seleccionat 1361 que semblen codificar alguna proteïna d'almenys 150 aminoàcids. S'ha realitzat ja un primer BLASTP amb les 1361 proteïnes predites, i la major part d'elles han trobat alguna homologia a la base de dades Swissprot.

Ara ens fixarem en les 57 d'aquelles 1361 proteïnes el millor resultat en BLASTP de les quals havia obtingut un valor E major que 0.1 (però no major que 1). És a dir, són proteïnes que se semblen poc a les ja presents a la base de dades Swissprot. El nostre objectiu serà determinar si mitjançant cerques de PSI-BLAST, més sensibles que el BLASTP, podem assignar estes proteïnes a algun grup de proteïnes relacionades remotament (o parcial), que ens puguen donar alguna pista de la seua funció.

L'arxiu `badHits.fa` és on es troben les 57 proteïnes predites d'*Ips typographus* seleccionades.

In [ ]:
# Si no açò no funciona, utilitza la consola per instal·lar blast.
system2(command = 'conda',
        args = c('install', '-c', 'bioconda', '-y', 'blast'),
        stdout = TRUE)

In [ ]:
system2(command = 'tar', args = c('-xzf', 'taxdb.tar.gz'), wait = TRUE)

In [ ]:
system2('psiblast',
        args = c('-query', 'badHits.fa',
                 '-db', 'swissprot',
                 '-out', 'psiblast.out',
                 '-outfmt', '7',
                 '-evalue', '1',
                 '-inclusion_ethresh', '0.7',
                 '-num_iterations', '5'))

## Exercicis

1. Corregeix si cal el codi anterior i executa'l.
2. Revisa els resultats i determina si hi ha proteïnes per a les quals encara no hem trobat homoglogies.
3. Pensa i discuteix si els resultats del PSI-BLAST ens poden ajudar a determinar la funció d'una proteïna desconeguda, i en quins casos.
4. Pensa i comprova què passa (i per a què serviria) canviar els llindar de valor E, `-evalue` i `-inclusion_ethresh`.

## Anàlisi individual

Hem executat el PSI-BLAST per a les 57 proteïnes d'una vegada, i els resultats han anat a parar tots al mateix arxiu. Tanmateix, podria ser més fàcil o convenient processar els resultats individuals de cada *query*. Una possibilitat és crear un arxiu fasta diferent per a cada una de les 57 proteïnes que estem analitzant. En R podem utilitzar la funció `write.fasta()` del paquet `seqinr`:

In [ ]:
library(seqinr)
library(Biostrings)

Les57 <- read.fasta('badHits.fa')
names(Les57)
write.fasta(Les57[['GACR01008940.1']],
            names = 'GACR01008940.1',
            file.out = 'GACR01008940.fa')

El PSI-BLAST elabora, després de cada iteració, una *position-specific scoring matrix* (PSSM), que resumeix la informació de l'alineament de les proteïnes homòlogues trobades. D'aquesta forma, la cerca és més sensible que amb un BLASTP convencional. Podem guardar la PSSM de l'última ronda de PSI-BLAST per realitzar amb ella una cerca addicional després:

In [ ]:
system2('psiblast',
        args = c('-query', 'GACR01008940.fa',
                 '-db', 'swissprot',
                 '-out', 'z1.out',
                 '-outfmt', '7',
                 '-evalue', '1',
                 '-inclusion_ethresh', '0.7',
                 '-num_iterations', '4',
                 '-save_pssm_after_last_round',
                 '-out_pssm', 'GACR01008940.pssm'))



Comprova que ha aparegut l'arxiu `GACR01008940.pssm`, el qual podem utilitzar ara com a *query*:

In [ ]:
system2('psiblast',
        args = c('-in_pssm', 'GACR01008940.pssm',
                 '-db', 'swissprot',
                 '-out', 'last.out',
                 '-outfmt', '7',
                 '-evalue', '1'))

Un avantatge de realitzar l'últim pas de PSI-BLAST amb la PSSM és que obtenim una única taula de resultats, no una per cada iteració. Això facilita carregar els resultats finals en la sessió de R:

In [ ]:
resultat <- read.table('last.out',
                        col.names = c('query','subject','identity','length','mismatch',
                                      'gaps','qstart','qend','sstart','send','evalue','bitscore'),
                        stringsAsFactors = FALSE)
plot(resultat$evalue, type = 'b', log = 'y')


In [ ]:
plot(c(1,380), range(resultat$evalue), type = 'n', ylab = 'Valor E', xlab = 'Aminoácidos', log = 'y')
segments(resultat$qstart, resultat$evalue, resultat$qend, resultat$evalue)

## Exercicis

1. Si queda temps, repeteix aquesta última anàlisi per a qualsevol altra proteïna de les 57 originals.
2. Com ho faries per repetir el procés de forma automàtica amb totes les proteïnes?